## בניית מודל רגרסיה לוגיסטית - שיעור 4

![אינפוגרפיקה של רגרסיה לוגיסטית מול רגרסיה ליניארית](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[שאלון לפני השיעור](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### מבוא

בשיעור האחרון על רגרסיה, אחת מטכניקות ה-ML הקלאסיות הבסיסיות, נבחן את רגרסיה לוגיסטית. תשתמשו בטכניקה זו כדי לזהות דפוסים ולחזות קטגוריות בינאריות. האם הממתק הזה הוא שוקולד או לא? האם המחלה הזו מדבקת או לא? האם הלקוח הזה יבחר במוצר הזה או לא?

בשיעור זה תלמדו:

- טכניקות לרגרסיה לוגיסטית

✅ העמיקו את ההבנה שלכם בעבודה עם סוג זה של רגרסיה במודול [Learn](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## דרישות מקדימות

לאחר שעבדנו עם נתוני הדלעת, אנחנו כבר מספיק מכירים אותם כדי להבין שיש קטגוריה בינארית אחת שנוכל לעבוד איתה: `Color`.

בואו נבנה מודל רגרסיה לוגיסטית כדי לחזות, בהתבסס על משתנים מסוימים, *איזה צבע צפוי להיות לדלעת מסוימת* (כתום 🎃 או לבן 👻).

> למה אנחנו מדברים על סיווג בינארי בשיעור שמקושר לרגרסיה? רק מטעמי נוחות לשונית, שכן רגרסיה לוגיסטית היא [בעצם שיטת סיווג](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), אם כי מבוססת ליניארית. למדו על דרכים אחרות לסווג נתונים בקבוצת השיעורים הבאה.

לשיעור זה נצטרך את החבילות הבאות:

- `tidyverse`: [tidyverse](https://www.tidyverse.org/) הוא [אוסף של חבילות R](https://www.tidyverse.org/packages) שנועד להפוך את מדע הנתונים למהיר, קל ומהנה יותר!

- `tidymodels`: [מסגרת tidymodels](https://www.tidymodels.org/) היא [אוסף של חבילות](https://www.tidymodels.org/packages/) למידול ולמידת מכונה.

- `janitor`: [חבילת janitor](https://github.com/sfirke/janitor) מספקת כלים פשוטים לבחינה וניקוי של נתונים מלוכלכים.

- `ggbeeswarm`: [חבילת ggbeeswarm](https://github.com/eclarke/ggbeeswarm) מספקת שיטות ליצירת גרפים בסגנון beeswarm באמצעות ggplot2.

ניתן להתקין אותן כך:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

לחילופין, הסקריפט הבא בודק אם יש לכם את החבילות הנדרשות להשלמת מודול זה ומתקין אותן עבורכם במקרה שחסרות.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **הגדרת השאלה**

למטרותינו, נבטא זאת כבינארי: 'לבן' או 'לא לבן'. יש גם קטגוריה של 'מפוספס' במאגר הנתונים שלנו, אך יש מעט מקרים שלה, ולכן לא נשתמש בה. היא נעלמת בכל מקרה כשאנחנו מסירים ערכים חסרים מהמאגר.

> 🎃 עובדה מעניינת, לפעמים אנחנו קוראים לדלעות לבנות 'דלעות רפאים'. הן לא קלות לגילוף, ולכן הן פחות פופולריות מהכתומות, אבל הן נראות מגניבות! אז אפשר גם לנסח מחדש את השאלה שלנו כ: 'רפאים' או 'לא רפאים'. 👻

## **על רגרסיה לוגיסטית**

רגרסיה לוגיסטית שונה מרגרסיה ליניארית, שלמדתם עליה קודם, בכמה דרכים חשובות.

#### **סיווג בינארי**

רגרסיה לוגיסטית אינה מציעה את אותן תכונות כמו רגרסיה ליניארית. הראשונה מציעה תחזית על `קטגוריה בינארית` ("כתום או לא כתום") בעוד שהאחרונה מסוגלת לחזות `ערכים רציפים`, לדוגמה, בהתחשב במקור הדלעת ובזמן הקטיף, *כמה המחיר שלה יעלה*.

![אינפוגרפיקה מאת Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### סיווגים אחרים

ישנם סוגים נוספים של רגרסיה לוגיסטית, כולל מולטינומיאלית ואורדינלית:

- **מולטינומיאלית**, שכוללת יותר מקטגוריה אחת - "כתום, לבן ומפוספס".

- **אורדינלית**, שכוללת קטגוריות מסודרות, שימושית אם נרצה לסדר את התוצאות שלנו באופן לוגי, כמו הדלעות שלנו שמסודרות לפי מספר סופי של גדלים (מיני, קטן, בינוני, גדול, גדול מאוד, ענק).

![רגרסיה מולטינומיאלית מול אורדינלית](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **המשתנים לא חייבים להיות מתואמים**

זוכרים איך רגרסיה ליניארית עבדה טוב יותר עם משתנים מתואמים? רגרסיה לוגיסטית היא ההפך - המשתנים לא חייבים להיות מתואמים. זה מתאים לנתונים שלנו שיש להם מתאמים חלשים יחסית.

#### **צריך הרבה נתונים נקיים**

רגרסיה לוגיסטית תיתן תוצאות מדויקות יותר אם תשתמשו ביותר נתונים; מאגר הנתונים הקטן שלנו אינו אופטימלי למשימה זו, אז קחו זאת בחשבון.

✅ חשבו על סוגי הנתונים שיתאימו היטב לרגרסיה לוגיסטית

## תרגיל - ניקוי הנתונים

ראשית, נקו את הנתונים מעט, הסירו ערכים חסרים ובחרו רק חלק מהעמודות:

1. הוסיפו את הקוד הבא:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


תמיד אפשר להציץ ב-DataFrame החדש שלך, באמצעות הפונקציה [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) כפי שמוצג למטה:


In [ ]:
pumpkins_select %>% 
  glimpse()


בואו נאשר שאנחנו באמת עומדים לבצע בעיית סיווג בינארית:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### ויזואליזציה - תרשים קטגורי
עד עכשיו טענת שוב את נתוני הדלעת וניקית אותם כך שיישאר לך מאגר נתונים המכיל כמה משתנים, כולל צבע. בואו נבצע ויזואליזציה של מסגרת הנתונים במחברת באמצעות ספריית ggplot.

ספריית ggplot מציעה דרכים נהדרות להמחיש את הנתונים שלך. לדוגמה, ניתן להשוות את התפלגויות הנתונים עבור כל זן וצבע בתרשים קטגורי.

1. צור תרשים כזה באמצעות הפונקציה geombar, תוך שימוש בנתוני הדלעת שלנו, והגדרת מיפוי צבע עבור כל קטגוריית דלעת (כתום או לבן):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

על ידי התבוננות בנתונים, ניתן לראות כיצד נתוני הצבע קשורים לזן.

✅ בהתבסס על התרשים הקטגורי הזה, אילו חקירות מעניינות ניתן לדמיין?


### עיבוד נתונים: קידוד מאפיינים

מערך הנתונים שלנו על דלעות מכיל ערכים מסוג מחרוזת בכל העמודות שלו. עבודה עם נתונים קטגוריים היא אינטואיטיבית עבור בני אדם, אך לא עבור מכונות. אלגוריתמים של למידת מכונה עובדים טוב עם מספרים. זו הסיבה שקידוד הוא שלב חשוב מאוד בשלב עיבוד הנתונים, מכיוון שהוא מאפשר לנו להפוך נתונים קטגוריים לנתונים מספריים, מבלי לאבד מידע. קידוד טוב מוביל לבניית מודל טוב.

לקידוד מאפיינים יש שני סוגים עיקריים של מקודדים:

1. מקודד סדרתי (Ordinal encoder): מתאים למשתנים סדרתיים, שהם משתנים קטגוריים שבהם הנתונים עוקבים אחר סדר לוגי, כמו עמודת `item_size` במערך הנתונים שלנו. הוא יוצר מיפוי כך שכל קטגוריה מיוצגת על ידי מספר, שהוא הסדר של הקטגוריה בעמודה.

2. מקודד קטגוריאלי (Categorical encoder): מתאים למשתנים נומינליים, שהם משתנים קטגוריים שבהם הנתונים אינם עוקבים אחר סדר לוגי, כמו כל המאפיינים השונים מ-`item_size` במערך הנתונים שלנו. זהו קידוד מסוג one-hot, כלומר כל קטגוריה מיוצגת על ידי עמודה בינארית: המשתנה המקודד שווה ל-1 אם הדלעת שייכת לזן הזה ול-0 אחרת.

Tidymodels מספקת חבילה נוספת ומועילה: [recipes](https://recipes.tidymodels.org/) - חבילה לעיבוד מקדים של נתונים. נגדיר `recipe` שמציין שכל עמודות המנבאים צריכות להיות מקודדות למערך של מספרים שלמים, נשתמש ב-`prep` כדי להעריך את הכמויות והסטטיסטיקות הנדרשות לכל פעולה, ולבסוף נשתמש ב-`bake` כדי ליישם את החישובים על נתונים חדשים.

> בדרך כלל, recipes משמשת כמעבד מקדים למידול, שבו היא מגדירה אילו שלבים יש ליישם על מערך נתונים כדי להכין אותו למידול. במקרה כזה **מומלץ מאוד** להשתמש ב-`workflow()` במקום להעריך מתכון באופן ידני באמצעות prep ו-bake. נראה את כל זה בעוד רגע.
>
> עם זאת, כרגע אנו משתמשים ב-recipes + prep + bake כדי להגדיר אילו שלבים יש ליישם על מערך נתונים כדי להכין אותו לניתוח נתונים, ולאחר מכן לחלץ את הנתונים שעברו עיבוד עם השלבים שהוחלו.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ מהם היתרונות בשימוש ב-ordinal encoder עבור עמודת Item Size?

### ניתוח קשרים בין משתנים

כעת, לאחר שעיבדנו את הנתונים שלנו, נוכל לנתח את הקשרים בין המאפיינים לתווית כדי לקבל מושג עד כמה המודל יוכל לחזות את התווית בהתבסס על המאפיינים. הדרך הטובה ביותר לבצע ניתוח מסוג זה היא על ידי הצגת הנתונים בגרף.  
נשתמש שוב בפונקציה ggplot geom_boxplot_ כדי להמחיש את הקשרים בין Item Size, Variety ו-Color בגרף קטגורי. כדי להציג את הנתונים בצורה טובה יותר, נשתמש בעמודת Item Size המקודדת ובעמודת Variety הלא מקודדת.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### שימוש בגרף נחיל

מכיוון ש-Color הוא קטגוריה בינארית (לבן או לא), יש צורך ב'[גישה מיוחדת](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)' לצורך ויזואליזציה.

נסו להשתמש ב`גרף נחיל` כדי להציג את התפלגות הצבע ביחס ל-item_size.

נשתמש בחבילת [ggbeeswarm](https://github.com/eclarke/ggbeeswarm), שמספקת שיטות ליצירת גרפים בסגנון נחיל באמצעות ggplot2. גרפי נחיל הם דרך להציג נקודות שבדרך כלל היו חופפות, כך שהן מוצגות זו לצד זו במקום זאת.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


עכשיו, כשיש לנו מושג על הקשר בין הקטגוריות הבינאריות של צבע לבין קבוצת הגדלים הרחבה יותר, בואו נחקור רגרסיה לוגיסטית כדי לקבוע את הצבע הסביר של דלעת נתונה.

## בניית המודל שלך

בחר את המשתנים שברצונך להשתמש בהם במודל הסיווג שלך וחלק את הנתונים לסטים של אימון ובדיקה. [rsample](https://rsample.tidymodels.org/), חבילה ב-Tidymodels, מספקת תשתית לחלוקה ולקיחת דגימות מחדש של נתונים בצורה יעילה:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 אנחנו מוכנים כעת לאמן מודל על ידי התאמת מאפייני האימון לתווית האימון (צבע).

נתחיל ביצירת מתכון שמפרט את שלבי הקדם-עיבוד שצריך לבצע על הנתונים שלנו כדי להכין אותם למידול, כלומר: קידוד משתנים קטגוריים למספרים שלמים. בדיוק כמו `baked_pumpkins`, ניצור `pumpkins_recipe` אך לא נבצע `prep` ו-`bake`, מכיוון שזה ייכלל בתוך זרימת עבודה, כפי שתראו בעוד כמה שלבים.

יש לא מעט דרכים להגדיר מודל רגרסיה לוגיסטית ב-Tidymodels. ראו `?logistic_reg()`. לעת עתה, נגדיר מודל רגרסיה לוגיסטית באמצעות מנוע ברירת המחדל `stats::glm()`.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


עכשיו, כשיש לנו מתכון ומפרט מודל, אנחנו צריכים למצוא דרך לשלב אותם יחד בתוך אובייקט שיבצע תחילה עיבוד מקדים לנתונים (prep+bake מאחורי הקלעים), יתאים את המודל לנתונים שעברו עיבוד מקדים, וגם יאפשר פעילויות עיבוד לאחרי.

ב-Tidymodels, האובייקט הנוח הזה נקרא [`workflow`](https://workflows.tidymodels.org/) והוא מחזיק בצורה נוחה את רכיבי המידול שלך.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


לאחר שזרימת עבודה *הוגדרה*, ניתן `לאמן` מודל באמצעות הפונקציה [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html). זרימת העבודה תעריך מתכון ותעבד את הנתונים לפני האימון, כך שלא נצטרך לעשות זאת באופן ידני באמצעות prep ו-bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


המודל מציג את המקדמים שנלמדו במהלך האימון.

עכשיו, לאחר שאימנו את המודל באמצעות נתוני האימון, נוכל לבצע תחזיות על נתוני הבדיקה באמצעות [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). נתחיל בשימוש במודל כדי לחזות תוויות עבור קבוצת הבדיקה שלנו ואת ההסתברויות עבור כל תווית. כאשר ההסתברות גבוהה מ-0.5, התווית החזויה היא `WHITE`, אחרת היא `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


ממש נחמד! זה מספק תובנות נוספות על איך רגרסיה לוגיסטית עובדת.

### הבנה טובה יותר באמצעות מטריצת בלבול

השוואת כל תחזית לערך האמיתי שלה ("האמת הקרקעית") אינה דרך יעילה במיוחד לקבוע עד כמה המודל מנבא בצורה טובה. למרבה המזל, ל-Tidymodels יש עוד כמה טריקים בשרוול: [`yardstick`](https://yardstick.tidymodels.org/) - חבילה המשמשת למדידת יעילות של מודלים באמצעות מדדי ביצועים.

אחד ממדדי הביצועים הקשורים לבעיות סיווג הוא [`מטריצת בלבול`](https://wikipedia.org/wiki/Confusion_matrix). מטריצת בלבול מתארת עד כמה מודל סיווג מבצע את עבודתו. מטריצת בלבול מציגה כמה דוגמאות בכל קטגוריה סווגו בצורה נכונה על ידי המודל. במקרה שלנו, היא תראה לך כמה דלעות כתומות סווגו ככתומות וכמה דלעות לבנות סווגו כלבנות; מטריצת הבלבול גם תראה לך כמה סווגו לקטגוריות **שגויות**.

הפונקציה [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) מתוך yardstick מחשבת את הטבלה הזו של קטגוריות נצפות וקטגוריות מנובאות.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


בואו נפרש את מטריצת הבלבול. המודל שלנו מתבקש לסווג דלעות לשתי קטגוריות בינאריות, קטגוריה `לבן` וקטגוריה `לא-לבן`.

- אם המודל שלכם מנבא שדלעת היא לבנה והיא באמת שייכת לקטגוריה 'לבן', אנו קוראים לזה `חיובי אמיתי`, שמוצג על ידי המספר בפינה השמאלית העליונה.

- אם המודל שלכם מנבא שדלעת היא לא לבנה והיא באמת שייכת לקטגוריה 'לבן', אנו קוראים לזה `שלילי שגוי`, שמוצג על ידי המספר בפינה השמאלית התחתונה.

- אם המודל שלכם מנבא שדלעת היא לבנה והיא באמת שייכת לקטגוריה 'לא-לבן', אנו קוראים לזה `חיובי שגוי`, שמוצג על ידי המספר בפינה הימנית העליונה.

- אם המודל שלכם מנבא שדלעת היא לא לבנה והיא באמת שייכת לקטגוריה 'לא-לבן', אנו קוראים לזה `שלילי אמיתי`, שמוצג על ידי המספר בפינה הימנית התחתונה.

| אמת |
|:-----:|

|               |        |       |
|---------------|--------|-------|
| **ניבוי**     | לבן   | כתום  |
| לבן           | TP     | FP    |
| כתום          | FN     | TN    |

כפי שכנראה ניחשתם, עדיף שיהיה מספר גדול יותר של חיוביים אמיתיים ושליליים אמיתיים ומספר קטן יותר של חיוביים שגויים ושליליים שגויים, מה שמעיד על כך שהמודל מתפקד טוב יותר.

מטריצת הבלבול מועילה מכיוון שהיא מובילה למדדים נוספים שיכולים לעזור לנו להעריך טוב יותר את ביצועי מודל הסיווג. בואו נעבור על כמה מהם:

🎓 דיוק: `TP/(TP + FP)` מוגדר כחלק מהחיוביים שנחזו שהם באמת חיוביים. נקרא גם [ערך ניבוי חיובי](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 רגישות: `TP/(TP + FN)` מוגדרת כחלק מהתוצאות החיוביות מתוך מספר הדגימות שהיו באמת חיוביות. ידועה גם בשם `רגישות`.

🎓 סגוליות: `TN/(TN + FP)` מוגדרת כחלק מהתוצאות השליליות מתוך מספר הדגימות שהיו באמת שליליות.

🎓 דיוק כללי: `TP + TN/(TP + TN + FP + FN)` אחוז התוויות שנחזו בצורה מדויקת עבור דגימה.

🎓 מדד F: ממוצע משוקלל של דיוק ורגישות, כאשר הטוב ביותר הוא 1 והגרוע ביותר הוא 0.

בואו נחשב את המדדים האלה!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## הצגת עקומת ה-ROC של המודל הזה

בואו נעשה ויזואליזציה נוספת כדי לראות את מה שנקרא [`עקומת ROC`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


עקומות ROC משמשות לעיתים קרובות כדי לקבל מבט על הביצועים של מסווג במונחים של חיוביים אמיתיים לעומת חיוביים שגויים. עקומות ROC מציגות בדרך כלל את `True Positive Rate`/רגישות על ציר ה-Y, ואת `False Positive Rate`/1-ספציפיות על ציר ה-X. לכן, השיפוע של העקומה והמרחק בין קו האמצע לעקומה חשובים: המטרה היא עקומה שעולה במהירות ומתרחקת מקו האמצע. במקרה שלנו, ישנם חיוביים שגויים בהתחלה, ואז הקו עולה ומתרחק בצורה תקינה.

לבסוף, נשתמש ב-`yardstick::roc_auc()` כדי לחשב את השטח מתחת לעקומה (AUC). אחת הדרכים לפרש את ה-AUC היא כהסתברות שהמודל מדרג דוגמה חיובית אקראית גבוה יותר מדוגמה שלילית אקראית.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


התוצאה היא בערך `0.975`. מאחר ש-AUC נע בין 0 ל-1, אתם רוצים ציון גבוה, שכן מודל שמנבא בצורה מדויקת ב-100% יקבל AUC של 1; במקרה הזה, המודל הוא *די טוב*.

בשיעורים הבאים על סיווגים, תלמדו כיצד לשפר את ציוני המודל שלכם (כמו למשל להתמודד עם נתונים לא מאוזנים במקרה הזה).

## 🚀אתגר

יש עוד הרבה מה ללמוד על רגרסיה לוגיסטית! אבל הדרך הטובה ביותר ללמוד היא להתנסות. מצאו מערך נתונים שמתאים לניתוח מסוג זה ובנו מודל בעזרתו. מה למדתם? טיפ: נסו את [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) למציאת מערכי נתונים מעניינים.

## סקירה ולמידה עצמית

קראו את העמודים הראשונים של [המאמר הזה מסטנפורד](https://web.stanford.edu/~jurafsky/slp3/5.pdf) על שימושים מעשיים ברגרסיה לוגיסטית. חשבו על משימות שמתאימות יותר לאחד מסוגי הרגרסיה שלמדנו עד כה. מה יעבוד הכי טוב?



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
